In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 4.2 Data preparation step
import pandas as panda
import numpy as numpy
import seaborn as sns
from matplotlib import pyplot
data_fram = panda.read_csv("/content/drive/My Drive/MlF/fer2013.csv")
print(data_fram) #number with disgust emotion [35887 rows x 3 columns]
data_frame=data_fram.loc[data_fram['emotion'] != 1]
print(data_frame) #number without disgust emotion [35340 rows x 3 columns], we use it in model

In [ ]:
def convert_images_to_gray_scale():
    gray_images = data_frame.pixels.apply(convert_from_pixel_array_to_image_array)
    gray_images = numpy.stack(gray_images, axis=0)
    return gray_images

def convert_from_pixel_array_to_image_array(pixel_array):
    pixel_array = numpy.array(pixel_array.split(" "))
    pixel_array = pixel_array.reshape(48, 48, 1)
    pixel_array = pixel_array.astype("float32")
    return pixel_array

def split_data():
    X_train, X_valid, y_train, y_valid = train_test_split(
        gray_image, image_label,
        test_size=0.2, shuffle=True,
        random_state=42, stratify=image_label
    )
    return X_train, X_valid, y_train, y_valid

In [ ]:
gray_image = convert_images_to_gray_scale()

In [ ]:
# Making images compatible to the model
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
number = LabelEncoder()
# Convert the array to class vector, then converts this class vector to binary class matrix
image_label = np_utils.to_categorical(number.fit_transform(data_frame.emotion))

# Splitting Data
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = split_data()
# Normalize the image arrays using min-max normalization. For gray-scaled images min=0 and max=255
X_train = X_train / 255.
X_valid = X_valid / 255.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dropout, BatchNormalization, Activation
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from keras.utils import np_utils

def get_conv2D(filters, kernel_size):
    return Conv2D(
          filters=filters,
          kernel_size=kernel_size,
          input_shape=(48, 48, 1),
          activation='elu',
          padding='same',
          kernel_initializer='he_normal'
    )

model = Sequential()

model.add(get_conv2D(64, (5, 5)))
model.add(BatchNormalization())
model.add(get_conv2D(64, (5, 5)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(get_conv2D(128, (3, 3)))
model.add(BatchNormalization())
model.add(get_conv2D(128, (3, 3)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(get_conv2D(256, (3, 3)))
model.add(BatchNormalization())
model.add(get_conv2D(256, (3, 3)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.5))

model.add(Flatten())    
model.add(Dense(128, activation='elu', kernel_initializer='he_normal'))
model.add(BatchNormalization())
model.add(Dropout(0.6))
model.add(Dense(6, activation='softmax')) ###

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

train_datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.15,
    height_shift_range=0.15,
    shear_range=0.15,
    zoom_range=0.15,
    horizontal_flip=True,
)

train_datagen.fit(X_train)

history = model.fit_generator(
    train_datagen.flow(X_train, y_train, batch_size=32),
    validation_data=(X_valid, y_valid),
    steps_per_epoch=len(X_train) / 32,
    epochs=200,
    use_multiprocessing=True
)

In [ ]:
import seaborn as sns
from matplotlib import pyplot

sns.set()
fig = pyplot.figure(0, (12, 4))

ax = pyplot.subplot(1, 2, 1)
sns.lineplot(history.epoch, history.history['accuracy'], label='train')
sns.lineplot(history.epoch, history.history['val_accuracy'], label='valid')
pyplot.title('Accuracy')
pyplot.tight_layout()

ax = pyplot.subplot(1, 2, 2)
sns.lineplot(history.epoch, history.history['loss'], label='train')
sns.lineplot(history.epoch, history.history['val_loss'], label='valid')
pyplot.title('Loss')
pyplot.tight_layout()

pyplot.savefig('epoch_history_dcnn.png')
pyplot.show()

In [ ]:
# Draw a bar chart representing the count of each emotion
sns.countplot(df.emotion)
pyplot.show()